In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('gender_submission.csv')

In [18]:
print(train.shape)
print(test.shape)
print(sub.shape)

(891, 12)
(418, 11)
(418, 2)


In [230]:
from sklearn.ensemble import RandomForestClassifier

In [235]:
from sklearn.metrics import accuracy_score

In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          0 non-null      object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


In [89]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 36.0+ KB


In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


문자형 데이터 : Name , Sex , Cabin , Ticket , Embarked -> 숫자형 데이터로 바꿔줄 것

In [45]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex            891
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

train 데이터에서는 결측값이 Age가 177개, Cabin이 687개, Embarked에 2개가 있다. -> 합리적인 값으로 채워줘야 함

In [46]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

test 데이터에서는 결측값이 Age가 86개, Cabin이 327개, Fare에 1개가 있습니다.

# Sex

applymap 함수를 통해서, male 값을 1, female 값을 0인 수치형 데이터로 바꿔주겠습니다.

In [132]:
sex_mapping = {'male':1, 'female':0}
train[['Sex']]= train[['Sex']].applymap(sex_mapping.get)
test[['Sex']]= test[['Sex']].applymap(sex_mapping.get)

# Name

Name에서 얻을 수 있는 정보는 Mr, Miss, Mrs 정도인데 이는 Age나 Sex를 통해 얻어낼 수 있는 정보라고 판단, 다른 데이터를 전처리하고 나서
다시 해보기로 했습니다.

# Embarked

Embarked는 승객이 어느 항구에서 탔는지를 나타냅니다. train 데이터에서는 결측치가 2개가 있고, test 데이터에서는 결측치가 없었습니다.

In [48]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

train 데이터의 Embarked 열에는 S값이 가장 많으므로, 2개의 결측치를 S로 채워주겠습니다.

In [79]:
train['Embarked'].fillna('S', inplace=True)

In [50]:
train['Embarked'].isnull().sum()

0

applymap 함수를 통해서, S 값을 0, C 값을 1, Q 값을 2인 수치형 데이터로 바꿔주겠습니다.

In [80]:
embarked_mapping = {'S':0, 'C':1, 'Q':2}
train[['Embarked']]= train[['Embarked']].applymap(embarked_mapping.get)
test[['Embarked']] = test[['Embarked']].applymap(embarked_mapping.get)

In [91]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",None,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",None,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",None,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",None,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",None,35.0,0,0,373450,8.0500,NaN,0


# Cabin

train 데이터 Cabin 열의 데이터를 보면, 알 수 있는 점이 2가지 있었습니다.
1. 앞에 있는 영어가 의미가 있어 보인다.
2. 결측치가 687개로 매우 많다.

1번은 영어만 가져온 후, 이를 수치형으로 변환했습니다.
2번은 Cabin과 관련성이 있는 열은 Pclass이고, 이유는 Pclass가 객실 등급을 의미하며, 결국 같은 등급의 객실의 객실번호인 Cabin이 비슷할거라고 생각했기 때문입니다. 우선 1번 과정을 거치고, Pclass로 그룹화한뒤, 각각의 객실의 중위값으로 변환시켜줬습니다.

Cabin 열이 생존자를 예상하는데 도움이 되는 열인지 확신이 들지 않아서, 이후에 Feature로 넣어보기전 / 넣어본후를 비교해볼 예정입니다.

In [93]:
train['Cabin'].str[:1].value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [121]:
train[['Cabin']] = train2[['Cabin']]
test[['Cabin']] = test2[['Cabin']]

In [122]:
train['Cabin'] = train['Cabin'].str[:1]
test['Cabin']= test['Cabin'].str[:1]

In [123]:
cabin_mapping = {'A':0, 'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7}
train[['Cabin']] = train[['Cabin']].applymap(cabin_mapping.get)
test[['Cabin']] = test[['Cabin']].applymap(cabin_mapping.get)

In [124]:
train['Cabin'] = train.groupby('Pclass')['Cabin'].transform('median')
test['Cabin'] = test.groupby('Pclass')['Cabin'].transform('median')

# Ticket

Ticket 열은 생존과 상관이 없다고 생각했고, 사실 수치형 데이터로 어떻게 변환해야할지 안 떠올라서 버렸습니다.

In [130]:
train.drop('Ticket',axis=1,inplace=True)
test.drop('Ticket',axis=1, inplace=True)

In [147]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,7.2500,5.0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,71.2833,2.0,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,7.9250,5.0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,53.1000,2.0,0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,8.0500,5.0,0


In [148]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,7.8292,5.0,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,7.0000,5.0,0
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,9.6875,5.0,2
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,8.6625,5.0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,12.2875,5.0,0


# Age & Name

train 데이터에서 Age 열의 결측치는 177개, test 데이터에서 Age 열의 결측치는 86개 있었습니다.

Name 열을 보면 Mr, Mrs, Miss, Master 등이 있는데, 이런 호칭들로 대략의 Age를 예상이 가능하다고 판단했습니다.

그래서, Name 열의 호칭을 전처리를 통해 정리하기로 했습니다.

이 과정은 구글링을 통해 해결했습니다.

In [151]:
train['Name'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Name'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

train['Name'] = train['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')


In [152]:
train['Name'] = train['Name'].replace('Mlle', 'Miss')
train['Name'] = train['Name'].replace('Ms', 'Miss')
train['Name'] = train['Name'].replace('Mme', 'Mrs')

In [172]:
test['Name'] = test['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

test['Name'] = test['Name'].replace('Mlle', 'Miss')
test['Name'] = test['Name'].replace('Ms', 'Miss')
test['Name'] = test['Name'].replace('Mme', 'Mrs')


In [162]:
train['Name'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Other      23
Name: Name, dtype: int64

In [174]:
test['Name'].value_counts()

Mr        240
Miss       79
Mrs        72
Master     21
Other       6
Name: Name, dtype: int64

Name 열의 데이터를 Mr , Miss, Mrs, Master, Other 로 변환해줬습니다. 이후에, 수치형데이터로 변환해주겠습니다.

In [164]:
name_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5}

In [165]:
train[['Name']] = train[['Name']].applymap(name_mapping.get)
test[['Name']] = test[['Name']].applymap(name_mapping.get)

In [166]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,1,0,3,1,1,22.0,1,0,7.2500,5.0,0,Mr
1,2,1,1,3,0,38.0,1,0,71.2833,2.0,1,Mrs
2,3,1,3,2,0,26.0,0,0,7.9250,5.0,0,Miss
3,4,1,1,3,0,35.0,1,0,53.1000,2.0,0,Mrs
4,5,0,3,1,1,35.0,0,0,8.0500,5.0,0,Mr


In [176]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,892,3,1,1,34.5,0,0,7.8292,5.0,2,Mr
1,893,3,3,0,47.0,1,0,7.0000,5.0,0,Mrs
2,894,2,1,1,62.0,0,0,9.6875,5.0,2,Mr
3,895,3,1,1,27.0,0,0,8.6625,5.0,0,Mr
4,896,3,3,0,22.0,1,1,12.2875,5.0,0,Mrs


# Age 2

이전까지 Age의 결측치의 갯수가 많았고, 이 결측치들을 어떻게 채워줄까 고민해봤습니다. 이 과정에서 Name과의 연관성을 확인했습니다.

구글링을 통해, Name으로 그룹화한 값의 중위값으로 결측치를 채워주는 것이 좋다는 글을 읽었고 실행해봤습니다.

In [179]:
train["Age"].fillna(train.groupby("Name")["Age"].transform("median"), inplace=True)
test["Age"].fillna(test.groupby("Name")["Age"].transform("median"), inplace=True)

어떤 글에서 읽었는지는 기억이 나지 않지만, 데이터분석시에 Pclass , Name, Sex, Embarked , Cabin과 같은 경우 mapping을 통해

전처리를 한 열들과 / Age와 Fare 와 같이 수치형데이터를 그대로 머신러닝에 적용시키면, 결과값이 좋지 않다는 글을 읽었습니다.

그래서, 이후에 Age와 Fare의 경우도 이전에 문자형데이터를 수치형데이터로 변환시켰던것과 비슷하게, Binning을 통하여 변환시켜줬습니다.

In [184]:
# Train
train.loc[ train['Age'] <= 10, 'Age_clean'] = 0
train.loc[(train['Age'] > 10) & (train['Age'] <= 16), 'Age_clean'] = 1
train.loc[(train['Age'] > 16) & (train['Age'] <= 20), 'Age_clean'] = 2
train.loc[(train['Age'] > 20) & (train['Age'] <= 26), 'Age_clean'] = 3
train.loc[(train['Age'] > 26) & (train['Age'] <= 30), 'Age_clean'] = 4
train.loc[(train['Age'] > 30) & (train['Age'] <= 36), 'Age_clean'] = 5
train.loc[(train['Age'] > 36) & (train['Age'] <= 40), 'Age_clean'] = 6
train.loc[(train['Age'] > 40) & (train['Age'] <= 46), 'Age_clean'] = 7
train.loc[(train['Age'] > 46) & (train['Age'] <= 50), 'Age_clean'] = 8
train.loc[(train['Age'] > 50) & (train['Age'] <= 60), 'Age_clean'] = 9
train.loc[ train['Age'] > 60, 'Age_clean'] = 10

# Test
test.loc[ test['Age'] <= 10, 'Age_clean'] = 0
test.loc[(test['Age'] > 10) & (test['Age'] <= 16), 'Age_clean'] = 1
test.loc[(test['Age'] > 16) & (test['Age'] <= 20), 'Age_clean'] = 2
test.loc[(test['Age'] > 20) & (test['Age'] <= 26), 'Age_clean'] = 3
test.loc[(test['Age'] > 26) & (test['Age'] <= 30), 'Age_clean'] = 4
test.loc[(test['Age'] > 30) & (test['Age'] <= 36), 'Age_clean'] = 5
test.loc[(test['Age'] > 36) & (test['Age'] <= 40), 'Age_clean'] = 6
test.loc[(test['Age'] > 40) & (test['Age'] <= 46), 'Age_clean'] = 7
test.loc[(test['Age'] > 46) & (test['Age'] <= 50), 'Age_clean'] = 8
test.loc[(test['Age'] > 50) & (test['Age'] <= 60), 'Age_clean'] = 9
test.loc[ test['Age'] > 60, 'Age_clean'] = 10

In [186]:
train[['Age']] = train[['Age_clean']]
test[['Age']] = test[['Age_clean']]

In [188]:
train.drop('Age_clean',axis=1,inplace=True)
test.drop('Age_clean',axis=1,inplace=True)

# Fare

Fare도 Age열과 같이 Binnig을 통하여 변환시키고자 했습니다.

나이의 경우 10대,20대 하니까 위와 같이 범위를 나누는 것에 납득을했는데, Fare의 경우 고민을 해봤습니다.

구글링 결과 pandas의 qcut함수를 사용한 글을 봐서 적용해봤습니다.

In [197]:
train['Fare'] = pd.qcut(train['Fare'], 5)
test['Fare'] = pd.qcut(test['Fare'], 5)

train['Fare'].value_counts()

(7.854, 10.5]        184
(21.679, 39.688]     180
(-0.001, 7.854]      179
(39.688, 512.329]    176
(10.5, 21.679]       172
Name: Fare, dtype: int64

In [198]:
train['Fare'] = train['Fare'].astype('category').cat.codes
test['Fare'] = test['Fare'].astype('category').cat.codes

train['Fare'].value_counts()

1    184
3    180
0    179
4    176
2    172
Name: Fare, dtype: int64

이제 남은 전처리 : SibSp , Parch 입니다.

SibSp : 동반한 형제자매와 배우자의 수

Parch : 동반한 부모와 자식의 수를 의미합니다.

이 두 열을 통해, 혼자인지, 동반한 가족이 있는지로 분류해서 데이터분석에 이용해야겠다고 생각했습니다.

혼자면 탈출하기 편했을거고, 동반한 가족이 있는경우 탈출하기 어려웠을 것입니다.

이를 위해 Family라는 열을 새로 만들어줬습니다.

In [206]:
train['Family'] = 1 + train['SibSp'] + train['Parch']
test['Family'] = 1 + test['SibSp'] + test['Parch']

In [211]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Family
0,1,0,3,1,1,3.0,1,0,0,5.0,0,2
1,2,1,1,3,0,6.0,1,0,4,2.0,1,2
2,3,1,3,2,0,3.0,0,0,1,5.0,0,1
3,4,1,1,3,0,5.0,1,0,4,2.0,0,2
4,5,0,3,1,1,5.0,0,0,1,5.0,0,1


# 데이터 준비

In [220]:
X_train = train.drop('Survived',axis=1)

In [232]:
X_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Family
0,3,1,1,3.0,1,0,0,5.0,0,2
1,1,3,0,6.0,1,0,4,2.0,1,2
2,3,2,0,3.0,0,0,1,5.0,0,1
3,1,3,0,5.0,1,0,4,2.0,0,2
4,3,1,1,5.0,0,0,1,5.0,0,1


In [221]:
Y_train = train['Survived']

In [236]:
test3.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [227]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Family
0,3,1,1,5.0,0,0,1,5.0,2,1
1,3,3,0,8.0,1,0,0,5.0,0,2
2,2,1,1,10.0,0,0,1,5.0,2,1
3,3,1,1,4.0,0,0,1,5.0,0,1
4,3,3,0,3.0,1,1,2,5.0,0,3


In [229]:
X_train.shape , Y_train.shape ,X_test.shape

((891, 10), (891,), (418, 10))

Pclass, Name, Sex, Age, SibSp, Parch, Fare, Cabin, Embarked, Family 를 feature로 설정하고,
RandomForestClassifier를 이용해서 정확도를 예측해봤습니다.

In [247]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
accuracy = round(random_forest.score(X_train, Y_train) * 100, 2)
accuracy

91.02

In [239]:
sub['Survived'] = Y_pred

In [245]:
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


# 하이퍼 파라미터 튜닝

저는 RandomForest를 이용했기 때문에, 이와 관련된 하이퍼 파라미터 튜닝을 찾아봤습니다.

n_estimator : forest size, 의사결정나무의 개수, 클수록 성능은 좋아질 수 있지만 시간이 오래 걸림

criterion : 'gini', 'entropy' 중 선택 가능, 기본 값은 'gini'이며, 의사 결정 나무를 학습하는 과정에서 기준 선정

max_depth : 과적합을 방지하기 위하여 의사결정나무의 depth 최대 값을 제한

min_samples_split : split을 끝낼 node의 sample 최소 갯수 선정, 클수록 과적합이 방지되지만, 너무 크면 split이 제대로 되지 않을 수 있음

min_samples_leaf : leaf node의 최소 sample 개수를 의미

n_jobs : -1로 설정시 컴퓨팅 프로세서 최대한 활용

random_state : 임의의 정수로 설정시 실행시마다 결과가 바뀌지 않고 고정됨

저는 이중에서 n_estimator만 100으로 설정하고, 다른값들은 건들지 않았습니다.

다른 타이타닉 코드들을 보면 대부분 n_estimator 를 50 혹은 100으로 설정해서 100으로 설정해서 진행했습니다.

max_depth를 6 정도로 설정하고 실행한 글도 있어서 적용해봤습니다.

91.02 -> 86.2 로 떨어져서 max_depth의 default값을 찾아봤는데 None이여서

처음에 나온 91.02라는 정확도는 과적합 때문에 높은 수치가 나왔다고 예상해봤습니다.